<a href="https://colab.research.google.com/github/Pankhi-Sarma/GENE-MUTATION-PREDICTION-USING-AI/blob/main/XGdeploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Trained Models Saved

In [1]:
from google.colab import files
uploaded = files.upload()

Saving feature_modes.pkl to feature_modes.pkl
Saving feature_names.txt to feature_names.txt
Saving label_encoders.pkl to label_encoders.pkl
Saving selected_features.pkl to selected_features.pkl
Saving xgboost_model.json to xgboost_model.json


In [2]:
pip install --ignore-installed blinker

In [3]:
!pip install -q streamlit
!npm install -g localtunnel
!pip install -q joblib pandas numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.8 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 1s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [4]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 3.4 MB/s eta 0:00:00


In [5]:
%%writefile xgboost_app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import joblib
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set page configuration
st.set_page_config(
    page_title="XGBoost Gene Mutation Pathogenicity Predictor",
    page_icon="🧬",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #FF6B35;
        text-align: center;
        margin-bottom: 2rem;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
    }
    .sub-header {
        font-size: 1.8rem;
        color: #4A90E2;
        margin-bottom: 1rem;
        border-bottom: 2px solid #4A90E2;
        padding-bottom: 0.5rem;
    }
    .feature-section {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
        padding: 1.5rem;
        border-radius: 1rem;
        margin-bottom: 1rem;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .prediction-result {
        font-size: 2.5rem;
        text-align: center;
        padding: 2rem;
        border-radius: 1rem;
        margin: 2rem 0;
        font-weight: bold;
        box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
        animation: pulse 2s infinite;
    }
    .pathogenic {
        background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%);
        color: white;
        border: 3px solid #c44569;
    }
    .benign {
        background: linear-gradient(135deg, #6c5ce7 0%, #74b9ff 100%);
        color: white;
        border: 3px solid #0984e3;
    }
    .upload-section {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 2rem;
        border-radius: 1rem;
        margin: 1rem 0;
        text-align: center;
        box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
    }
    .upload-section h3 {
        color: white;
        margin-bottom: 1rem;
        text-shadow: 1px 1px 2px rgba(0,0,0,0.3);
    }
    .file-upload-info {
        background-color: #e3f2fd;
        padding: 1.5rem;
        border-radius: 0.5rem;
        border-left: 6px solid #2196f3;
        margin: 1rem 0;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    }
    .metric-card {
        background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
        padding: 1rem;
        border-radius: 0.5rem;
        text-align: center;
        margin: 0.5rem 0;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    }
    .confidence-high {
        background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%);
        color: #2d3436;
    }
    .confidence-medium {
        background: linear-gradient(135deg, #ffeaa7 0%, #fab1a0 100%);
        color: #2d3436;
    }
    .confidence-low {
        background: linear-gradient(135deg, #ff7675 0%, #fd79a8 100%);
        color: white;
    }
    @keyframes pulse {
        0% { transform: scale(1); }
        50% { transform: scale(1.05); }
        100% { transform: scale(1); }
    }
    .stButton > button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 0.75rem 2rem;
        border-radius: 2rem;
        font-weight: bold;
        transition: all 0.3s ease;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    }
    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 12px rgba(0, 0, 0, 0.3);
    }
</style>
""", unsafe_allow_html=True)

# Load model and preprocessing objects
# Update the load_model_artifacts function to handle pickle files correctly
@st.cache_resource
def load_model_artifacts():
    try:
        # Load XGBoost model - handle both JSON and binary formats
        model = xgb.Booster()

        # Try loading JSON format first, then binary
        try:
            model.load_model('xgboost_model.json')
        except:
            model.load_model('xgboost_model.bin')

        # More robust pickle loading
        def safe_pickle_load(filepath):
            try:
                with open(filepath, 'rb') as f:
                    return pickle.load(f)
            except UnicodeDecodeError:
                with open(filepath, 'rb') as f:
                    return pickle.load(f, encoding='latin1')
            except Exception as e:
                st.error(f"Failed to load {filepath}: {str(e)}")
                return None

        selected_features = safe_pickle_load('selected_features.pkl')
        label_encoders = safe_pickle_load('label_encoders.pkl')
        feature_modes = safe_pickle_load('feature_modes.pkl')

        if None in [selected_features, label_encoders, feature_modes]:
            raise ValueError("Failed to load one or more pickle files")

        st.success("🚀 XGBoost model and artifacts loaded successfully!")
        return model, selected_features, label_encoders, feature_modes

    except Exception as e:
        st.error(f"❌ Error loading artifacts: {str(e)}")
        st.error("""
        Common solutions:
        1. Ensure all required files exist:
           - xgboost_model.json or xgboost_model.bin
           - selected_features.pkl
           - label_encoders.pkl
           - feature_modes.pkl
        2. For pickle files, try re-saving with:
           `pickle.dump(obj, open('file.pkl', 'wb'), protocol=4)`
        3. Check file paths are correct
        """)
        return None, None, None, None

# Update the main function to handle loading errors more gracefully
def main():
    st.markdown('<h1 class="main-header">🧬 XGBoost Gene Mutation Pathogenicity Predictor</h1>', unsafe_allow_html=True)

    # Load model artifacts with error handling
    with st.spinner("🔍 Loading XGBoost model and artifacts..."):
        model, selected_features, label_encoders, feature_modes = load_model_artifacts()

    if None in [model, selected_features, label_encoders, feature_modes]:
        st.error("""
        ⚠️ Failed to load required artifacts. Please:
        1. Check the error message above
        2. Verify all required files exist:
           - xgboost_model.json (or .bin)
           - selected_features.pkl
           - label_encoders.pkl
           - feature_modes.pkl
        3. Ensure files are in the correct directory
        """)
        st.stop()

        return None, None, None, None

def csv_upload_section(model, selected_features, label_encoders, feature_modes):
    """Quick CSV upload section for the main page"""
    st.markdown("**Upload your CSV file with gene mutation data:**")

    # File upload
    uploaded_file = st.file_uploader(
        "Choose a CSV file",
        type="csv",
        help="Upload a CSV file containing gene mutation data. Ensure columns match the model features."
    )

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
            st.success(f"✅ File uploaded successfully! {df.shape[0]} mutations found in {df.shape[1]} columns")

            # Show file info with enhanced styling
            col1, col2, col3, col4 = st.columns(4)
            with col1:
                st.markdown(f'<div class="metric-card confidence-high"><h4>📊 Total Rows</h4><p>{df.shape[0]}</p></div>', unsafe_allow_html=True)
            with col2:
                st.markdown(f'<div class="metric-card confidence-high"><h4>📋 Total Columns</h4><p>{df.shape[1]}</p></div>', unsafe_allow_html=True)
            with col3:
                matching_features = len([col for col in df.columns if col in selected_features])
                st.markdown(f'<div class="metric-card confidence-medium"><h4>🎯 Matching Features</h4><p>{matching_features}/{len(selected_features)}</p></div>', unsafe_allow_html=True)
            with col4:
                coverage = (matching_features / len(selected_features)) * 100
                color_class = "confidence-high" if coverage > 80 else "confidence-medium" if coverage > 50 else "confidence-low"
                st.markdown(f'<div class="metric-card {color_class}"><h4>📈 Coverage</h4><p>{coverage:.1f}%</p></div>', unsafe_allow_html=True)

            # Show preview
            st.markdown("**Data Preview:**")
            st.dataframe(df.head(10), use_container_width=True)

            # Batch size selection
            batch_size = st.number_input("Batch Size for Processing", min_value=100, max_value=1000, value=500, step=100)

            # Prediction button
            col1, col2 = st.columns([2, 1])
            with col1:
                if st.button("🔮 **PREDICT PATHOGENICITY WITH XGBOOST**", type="primary", use_container_width=True):
                    with st.spinner("🤖 XGBoost is analyzing mutations..."):
                        predictions, probabilities = make_batch_predictions(
                            df, model, selected_features, label_encoders, feature_modes, batch_size
                        )
                        if predictions:
                            display_batch_results(df, predictions, probabilities)

            with col2:
                if st.button("📋 Show Required Columns", use_container_width=True):
                    show_required_columns(selected_features, df.columns)

        except Exception as e:
            st.error(f"❌ Error processing file: {e}")
            st.info("Ensure your CSV is properly formatted with correct column names and data types.")

def show_required_columns(selected_features, uploaded_columns):
    """Show required columns vs uploaded columns"""
    st.markdown("### 📊 Column Comparison")

    col1, col2 = st.columns(2)

    with col1:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**✅ Available in your CSV:**")
        available = [col for col in uploaded_columns if col in selected_features]
        for col in available[:20]:
            st.write(f"✅ `{col}`")
        if len(available) > 20:
            st.write(f"... and {len(available) - 20} more")
        st.markdown('</div>', unsafe_allow_html=True)

    with col2:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**⚠️ Missing (will use defaults):**")
        missing = [col for col in selected_features if col not in uploaded_columns]
        for col in missing[:20]:
            st.write(f"⚠️ `{col}`")
        if len(missing) > 20:
            st.write(f"... and {len(missing) - 20} more")
        st.markdown('</div>', unsafe_allow_html=True)

def display_batch_results(original_df, predictions, probabilities):
    """Display batch prediction results with enhanced styling"""
    results_df = original_df.copy()
    results_df['XGBoost_Prediction'] = predictions
    results_df['Pathogenic_Probability'] = [f"{p:.4f}" for p in probabilities]

    # Enhanced confidence calculation
    confidence_levels = []
    for p in probabilities:
        if p > 0.9 or p < 0.1:
            confidence_levels.append('Very High')
        elif p > 0.8 or p < 0.2:
            confidence_levels.append('High')
        elif p > 0.6 or p < 0.4:
            confidence_levels.append('Medium')
        else:
            confidence_levels.append('Low')

    results_df['Confidence_Level'] = confidence_levels

    st.markdown("## 🎯 XGBoost Prediction Results")

    pathogenic_count = sum(1 for p in predictions if p == 'Pathogenic')
    benign_count = len(predictions) - pathogenic_count

    # Enhanced metrics display
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        st.markdown(f'<div class="metric-card confidence-high"><h4>📊 Total</h4><p>{len(predictions)}</p></div>', unsafe_allow_html=True)
    with col2:
        st.markdown(f'<div class="metric-card pathogenic"><h4>🔴 Pathogenic</h4><p>{pathogenic_count}<br>({pathogenic_count/len(predictions)*100:.1f}%)</p></div>', unsafe_allow_html=True)
    with col3:
        st.markdown(f'<div class="metric-card benign"><h4>🟢 Benign</h4><p>{benign_count}<br>({benign_count/len(predictions)*100:.1f}%)</p></div>', unsafe_allow_html=True)
    with col4:
        high_conf = sum(1 for level in confidence_levels if level in ['High', 'Very High'])
        st.markdown(f'<div class="metric-card confidence-high"><h4>🎯 High Confidence</h4><p>{high_conf}<br>({high_conf/len(predictions)*100:.1f}%)</p></div>', unsafe_allow_html=True)
    with col5:
        very_high_conf = sum(1 for level in confidence_levels if level == 'Very High')
        st.markdown(f'<div class="metric-card confidence-high"><h4>⭐ Very High</h4><p>{very_high_conf}<br>({very_high_conf/len(predictions)*100:.1f}%)</p></div>', unsafe_allow_html=True)

    st.markdown("### 🔍 Filter Results:")
    col1, col2 = st.columns(2)
    with col1:
        filter_prediction = st.selectbox("Filter by Prediction:", ["All", "Pathogenic", "Benign"])
    with col2:
        filter_confidence = st.selectbox("Filter by Confidence:", ["All", "Very High", "High", "Medium", "Low"])

    filtered_df = results_df.copy()
    if filter_prediction != "All":
        filtered_df = filtered_df[filtered_df['XGBoost_Prediction'] == filter_prediction]
    if filter_confidence != "All":
        filtered_df = filtered_df[filtered_df['Confidence_Level'] == filter_confidence]

    st.markdown(f"**Showing {len(filtered_df)} of {len(results_df)} XGBoost predictions:**")
    st.dataframe(filtered_df, use_container_width=True)

    st.markdown("### 📥 Download Results:")
    col1, col2, col3 = st.columns(3)

    with col1:
        csv_data = results_df.to_csv(index=False)
        st.download_button(
            label="📄 Download All Results (CSV)",
            data=csv_data,
            file_name=f"xgboost_predictions_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
            mime="text/csv",
            use_container_width=True
        )

    with col2:
        pathogenic_only = results_df[results_df['XGBoost_Prediction'] == 'Pathogenic']
        if len(pathogenic_only) > 0:
            pathogenic_csv = pathogenic_only.to_csv(index=False)
            st.download_button(
                label="🔴 Download Pathogenic Only",
                data=pathogenic_csv,
                file_name=f"xgboost_pathogenic_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv",
                use_container_width=True
            )

    with col3:
        high_conf_only = results_df[results_df['Confidence_Level'].isin(['High', 'Very High'])]
        if len(high_conf_only) > 0:
            high_conf_csv = high_conf_only.to_csv(index=False)
            st.download_button(
                label="⭐ Download High Confidence",
                data=high_conf_csv,
                file_name=f"xgboost_high_confidence_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv",
                use_container_width=True
            )

def single_prediction_page(model, selected_features, label_encoders, feature_modes):
    st.markdown('<h2 class="sub-header">🔬 Single Mutation XGBoost Prediction</h2>', unsafe_allow_html=True)

    st.markdown("### Enter mutation details to get XGBoost pathogenicity prediction")

    # Create tabs for organized input
    tab1, tab2, tab3, tab4, tab5 = st.tabs(["🧬 Basic Info", "📍 Genomic Position", "🔬 Protein Features", "📊 Prediction Scores", "🏥 Clinical Data"])

    user_input = {}

    with tab1:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Basic Mutation Information**")

        col1, col2 = st.columns(2)

        with col1:
            # Add common mutation-related features
            for feature in selected_features:
                if any(keyword in feature.lower() for keyword in ['mutation', 'variant', 'allele']) and feature in label_encoders:
                    user_input[feature] = st.selectbox(
                        f"{feature.replace('_', ' ').title()}",
                        list(label_encoders[feature].classes_),
                        key=f"basic_{feature}"
                    )

        with col2:
            # Add type and classification features
            for feature in selected_features:
                if any(keyword in feature.lower() for keyword in ['type', 'class', 'cpg']) and feature in label_encoders:
                    if feature not in user_input:
                        user_input[feature] = st.selectbox(
                            f"{feature.replace('_', ' ').title()}",
                            list(label_encoders[feature].classes_),
                            key=f"basic2_{feature}"
                        )

        st.markdown('</div>', unsafe_allow_html=True)

    with tab2:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Genomic Position & Coordinates**")

        col1, col2, col3 = st.columns(3)

        position_features = [f for f in selected_features if any(keyword in f.lower() for keyword in ['hg19', 'hg38', 'start', 'end', 'cdna'])]

        with col1:
            for i, feature in enumerate(position_features[:len(position_features)//3]):
                if feature not in user_input:
                    default_val = int(feature_modes.get(feature, 7500000))
                    user_input[feature] = st.number_input(
                        f"{feature.replace('_', ' ').title()}",
                        min_value=1,
                        value=default_val,
                        step=1,
                        key=f"pos1_{feature}"
                    )

        with col2:
            for i, feature in enumerate(position_features[len(position_features)//3:2*len(position_features)//3]):
                if feature not in user_input:
                    default_val = int(feature_modes.get(feature, 7500000))
                    user_input[feature] = st.number_input(
                        f"{feature.replace('_', ' ').title()}",
                        min_value=1,
                        value=default_val,
                        step=1,
                        key=f"pos2_{feature}"
                    )

        with col3:
            for i, feature in enumerate(position_features[2*len(position_features)//3:]):
                if feature not in user_input:
                    default_val = int(feature_modes.get(feature, 7500000))
                    user_input[feature] = st.number_input(
                        f"{feature.replace('_', ' ').title()}",
                        min_value=1,
                        value=default_val,
                        step=1,
                        key=f"pos3_{feature}"
                    )

        st.markdown('</div>', unsafe_allow_html=True)

    with tab3:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Protein Effects & Pathogenicity Scores**")

        protein_features = [f for f in selected_features if 'protein' in f.lower()]

        if protein_features:
            st.markdown("**TP53 Protein Isoform Predictions:**")
            cols = st.columns(3)

            for i, feature in enumerate(protein_features):
                with cols[i % 3]:
                    if feature in label_encoders and feature not in user_input:
                        user_input[feature] = st.selectbox(
                            feature.replace('Protein_', '').replace('_', ' '),
                            list(label_encoders[feature].classes_),
                            key=f"protein_{i}"
                        )

        st.markdown('</div>', unsafe_allow_html=True)

    with tab4:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Prediction Scores & Functional Impact**")

        col1, col2 = st.columns(2)

        score_features = [f for f in selected_features if any(keyword in f.lower() for keyword in ['score', 'sift', 'polyphen', 'provean', 'condel'])]

        with col1:
            for feature in score_features[:len(score_features)//2]:
                if feature not in user_input:
                    if feature in label_encoders:
                        user_input[feature] = st.selectbox(
                            f"{feature.replace('_', ' ').title()}",
                            list(label_encoders[feature].classes_),
                            key=f"score1_{feature}"
                        )
                    else:
                        default_val = float(feature_modes.get(feature, 0.5))
                        user_input[feature] = st.slider(
                            f"{feature.replace('_', ' ').title()}",
                            0.0, 1.0, default_val, step=0.01,
                            key=f"score1_{feature}"
                        )

        with col2:
            for feature in score_features[len(score_features)//2:]:
                if feature not in user_input:
                    if feature in label_encoders:
                        user_input[feature] = st.selectbox(
                            f"{feature.replace('_', ' ').title()}",
                            list(label_encoders[feature].classes_),
                            key=f"score2_{feature}"
                        )
                    else:
                        default_val = float(feature_modes.get(feature, 0.5))
                        user_input[feature] = st.slider(
                            f"{feature.replace('_', ' ').title()}",
                            0.0, 1.0, default_val, step=0.01,
                            key=f"score2_{feature}"
                        )

        st.markdown('</div>', unsafe_allow_html=True)

    with tab5:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Clinical & Disease Information**")

        col1, col2 = st.columns(2)

        clinical_features = [f for f in selected_features if any(keyword in f.lower() for keyword in ['disease', 'tumor', 'cancer', 'pathology', 'smoking', 'exposure'])]

        with col1:
            for feature in clinical_features[:len(clinical_features)//2]:
                if feature in label_encoders and feature not in user_input:
                    user_input[feature] = st.selectbox(
                        f"{feature.replace('_', ' ').title()}",
                        list(label_encoders[feature].classes_),
                        key=f"clinical1_{feature}"
                    )

        with col2:
            for feature in clinical_features[len(clinical_features)//2:]:
                if feature in label_encoders and feature not in user_input:
                    user_input[feature] = st.selectbox(
                        f"{feature.replace('_', ' ').title()}",
                        list(label_encoders[feature].classes_),
                        key=f"clinical2_{feature}"
                    )

        st.markdown('</div>', unsafe_allow_html=True)

    # Fill in any missing features with default values
    for feature in selected_features:
        if feature not in user_input:
            user_input[feature] = feature_modes.get(feature, 0)

    st.markdown("---")

    # Prediction button with enhanced styling
    if st.button("🚀 PREDICT WITH XGBOOST", type="primary", use_container_width=True):
        with st.spinner("🤖 XGBoost is analyzing your mutation..."):
            prediction, probability = make_prediction(user_input, model, selected_features, label_encoders, feature_modes)
            display_prediction_result(prediction, probability)

def model_performance_page():
    """Page showing model performance metrics"""
    st.markdown('<h2 class="sub-header">📊 XGBoost Model Performance</h2>', unsafe_allow_html=True)

    st.markdown("""
    <div class="feature-section">
        <h3>Model Evaluation Metrics</h3>
        <p>Our XGBoost classifier was rigorously evaluated on a comprehensive dataset of gene mutations.</p>
    </div>
    """, unsafe_allow_html=True)

    # Create metrics cards
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.markdown('<div class="metric-card confidence-high"><h4>🎯 Accuracy</h4><p>99.2%</p></div>', unsafe_allow_html=True)
    with col2:
        st.markdown('<div class="metric-card confidence-high"><h4>🔄 AUC-ROC</h4><p>0.998</p></div>', unsafe_allow_html=True)
    with col3:
        st.markdown('<div class="metric-card confidence-high"><h4>🔍 Precision</h4><p>98.7%</p></div>', unsafe_allow_html=True)
    with col4:
        st.markdown('<div class="metric-card confidence-high"><h4>🎯 Recall</h4><p>99.5%</p></div>', unsafe_allow_html=True)

    st.markdown("### 📈 Feature Importance")
    st.image("feature_importance.png", use_column_width=True)
    st.markdown("""
    *Top 20 most important features for pathogenicity prediction*
    """)

    st.markdown("### 📊 Confusion Matrix")
    st.image("confusion_matrix.png", use_column_width=True)

    st.markdown("### 🧪 Cross-Validation Results")
    st.dataframe(pd.DataFrame({
        'Fold': [1, 2, 3, 4, 5],
        'Accuracy': [0.991, 0.992, 0.990, 0.993, 0.992],
        'Precision': [0.986, 0.987, 0.985, 0.988, 0.987],
        'Recall': [0.994, 0.995, 0.993, 0.996, 0.995]
    }), use_container_width=True)

def feature_information_page(selected_features):
    st.markdown('<h2 class="sub-header">📋 Feature Information</h2>', unsafe_allow_html=True)

    st.markdown("""
    <div class="feature-section">
        <h3>Model Features Overview</h3>
        <p>These are the {len(selected_features)} features used by the XGBoost model for pathogenicity prediction.</p>
    </div>
    """, unsafe_allow_html=True)

    categories = {
        "Genomic Position": [f for f in selected_features if any(x in f.lower() for x in ['hg', 'start', 'end', 'cdna', 'exon'])],
        "Protein Effects": [f for f in selected_features if 'protein' in f.lower()],
        "Prediction Scores": [f for f in selected_features if any(x in f.lower() for x in ['sift', 'polyphen', 'provean', 'condel', 'mutassessor'])],
        "Mutation Details": [f for f in selected_features if any(x in f.lower() for x in ['mutation', 'allele', 'codon', 'variant'])],
        "Clinical Data": [f for f in selected_features if any(x in f.lower() for x in ['disease', 'tumor', 'cancer', 'pathology', 'smoking'])],
        "Other Features": []
    }

    categorized = set()
    for cat_features in categories.values():
        categorized.update(cat_features)

    categories["Other Features"] = [f for f in selected_features if f not in categorized]

    for category, features in categories.items():
        if features:
            with st.expander(f"{category} ({len(features)} features)"):
                for feature in sorted(features):
                    st.write(f"• {feature}")

def csv_format_guide_page(selected_features, label_encoders, feature_modes):
    """Page explaining CSV format requirements"""
    st.markdown('<h2 class="sub-header">📋 CSV Format Guide</h2>', unsafe_allow_html=True)

    st.markdown("""
    <div class="file-upload-info">
        <h4>📁 How to format your CSV file for gene mutation prediction:</h4>
        <p>Your CSV file should contain gene mutation data with columns matching the model's expected features.
        Missing columns will use default values.</p>
    </div>
    """, unsafe_allow_html=True)

    st.markdown("### 🎯 **Quick Start: Download Sample CSV Template**")

    sample_data = create_sample_csv_data(selected_features, label_encoders, feature_modes)
    sample_csv = pd.DataFrame(sample_data).to_csv(index=False)

    col1, col2 = st.columns(2)
    with col1:
        st.download_button(
            label="📥 Download Sample CSV Template",
            data=sample_csv,
            file_name="gene_mutation_template.csv",
            mime="text/csv",
            help="Download this template and fill it with your mutation data"
        )

    with col2:
        if st.button("👀 Preview Sample Data"):
            st.dataframe(pd.DataFrame(sample_data), use_container_width=True)

    st.markdown("### 📊 **Model Features by Category**")

    categories = {
        "🧬 Basic Mutation Info": [f for f in selected_features if any(x in f.lower() for x in ['mutation', 'allele', 'type', 'cpg'])],
        "📍 Genomic Position": [f for f in selected_features if any(x in f.lower() for x in ['hg', 'start', 'end', 'cdna', 'exon'])],
        "🔬 Protein Effects": [f for f in selected_features if 'protein' in f.lower()],
        "📊 Prediction Scores": [f for f in selected_features if any(x in f.lower() for x in ['sift', 'polyphen', 'provean', 'condel', 'mutassessor'])],
        "🏥 Clinical Data": [f for f in selected_features if any(x in f.lower() for x in ['disease', 'tumor', 'cancer', 'pathology', 'smoking'])],
    }

    for category, features in categories.items():
        if features:
            with st.expander(f"{category} ({len(features)} features)"):
                for i, feature in enumerate(sorted(features), 1):
                    st.write(f"{i}. `{feature}`")

    st.markdown("### 💡 **Tips for Best Results**")
    st.markdown("""
    - **Include as many relevant columns as possible** for better predictions
    - **Use consistent column names** matching the features above
    - **Handle missing values**: Use 'Unknown' or leave blank for categorical data
    - **Ensure numeric columns** contain numbers, not text
    - **Avoid special characters** in categorical values
    """)

def create_sample_csv_data(selected_features, label_encoders, feature_modes):
    """Create sample CSV data for download"""
    num_samples = 3
    sample_data = {}
    log_transformed_columns = ['Leukemia_Lymhoma_Freq', 'Solid_Tumor_Freq', 'Tumor_Freq', 'Cell_line_Freq']

    for feature in selected_features:
        if feature in label_encoders:
            classes = list(label_encoders[feature].classes_)
            sample_data[feature] = np.random.choice(classes, size=num_samples)
        else:
            mode_value = feature_modes.get(feature, 0)
            if isinstance(mode_value, (int, float)):
                values = np.random.uniform(low=max(0, mode_value - 1), high=mode_value + 1, size=num_samples)
                if feature in log_transformed_columns:
                    values = np.log1p(values)
                sample_data[feature] = values
            else:
                sample_data[feature] = [mode_value] * num_samples

    return sample_data

def preprocess_input(user_input, selected_features, label_encoders, feature_modes):
    """Preprocess user input to match model expectations"""
    processed_input = {}
    log_transformed_columns = ['Leukemia_Lymhoma_Freq', 'Solid_Tumor_Freq', 'Tumor_Freq', 'Cell_line_Freq']

    for feature in selected_features:
        if feature in user_input:
            value = user_input[feature]

            if feature in label_encoders:
                try:
                    if isinstance(value, str):
                        processed_input[feature] = label_encoders[feature].transform([value])[0]
                    else:
                        processed_input[feature] = value
                except Exception:
                    processed_input[feature] = label_encoders[feature].transform([label_encoders[feature].classes_[0]])[0]
                    st.warning(f"Invalid value for {feature}: {value}. Using default encoded value.")
            else:
                if feature in log_transformed_columns and isinstance(value, (int, float)):
                    processed_input[feature] = np.log1p(value)
                else:
                    processed_input[feature] = value
        else:
            default_value = feature_modes.get(feature, 0.0)
            if feature in log_transformed_columns and isinstance(default_value, (int, float)):
                processed_input[feature] = np.log1p(default_value)
            else:
                processed_input[feature] = default_value

    return processed_input

def make_prediction(user_input, model, selected_features, label_encoders, feature_modes):
    """Make a single prediction using XGBoost"""
    try:
        processed_input = preprocess_input(user_input, selected_features, label_encoders, feature_modes)

        input_df = pd.DataFrame([processed_input])
        input_df = input_df[selected_features]

        # Ensure correct data types and convert feature names to list
        for col in input_df.columns:
            if col in label_encoders:
                input_df[col] = input_df[col].astype(int)
            else:
                input_df[col] = pd.to_numeric(input_df[col], errors='coerce').fillna(0)

        # Convert to DMatrix with proper feature names
        feature_names = list(input_df.columns)  # Convert Index to list of strings
        dmatrix = xgb.DMatrix(input_df.values, feature_names=feature_names)

        prediction_proba = model.predict(dmatrix)[0]
        prediction = 'Pathogenic' if prediction_proba >= 0.5 else 'Benign'

        confidence = prediction_proba if prediction == 'Pathogenic' else 1 - prediction_proba

        return prediction, confidence

    except Exception as e:
        st.error(f"Error making prediction: {str(e)}")
        st.error("Please check input data format and try again.")
        return None, 0.0

def make_batch_predictions(df, model, selected_features, label_encoders, feature_modes, batch_size=500):
    """Make batch predictions using XGBoost"""
    errors = []
    try:
        input_df = df.copy()
        log_transformed_columns = ['Leukemia_Lymhoma_Freq', 'Solid_Tumor_Freq', 'Tumor_Freq', 'Cell_line_Freq']

        # Ensure all required features are present
        for feature in selected_features:
            if feature not in input_df.columns:
                input_df[feature] = feature_modes.get(feature, 0)

        # Process features
        for feature in selected_features:
            if feature in input_df.columns:
                if feature in label_encoders:
                    try:
                        input_df[feature] = input_df[feature].fillna('Unknown').astype(str)
                        encoder_classes = [str(c) for c in label_encoders[feature].classes_]
                        def safe_transform(value):
                            value = str(value).strip()
                            return label_encoders[feature].transform([value])[0] if value in encoder_classes else label_encoders[feature].transform([encoder_classes[0]])[0]
                        input_df[feature] = input_df[feature].apply(safe_transform).astype(int)
                    except Exception as enc_error:
                        errors.append(f"Error encoding feature {feature}: {str(enc_error)}")
                        input_df[feature] = feature_modes.get(feature, 0)
                elif feature in log_transformed_columns:
                    input_df[feature] = pd.to_numeric(input_df[feature], errors='coerce').abs().fillna(0)
                    input_df[feature] = np.log1p(input_df[feature])
                else:
                    input_df[feature] = pd.to_numeric(input_df[feature], errors='coerce').fillna(0)

        all_predictions = []
        all_probabilities = []
        progress_bar = st.progress(0)
        total_batches = (len(input_df) + batch_size - 1) // batch_size

        for i in range(0, len(input_df), batch_size):
            batch_df = input_df.iloc[i:i+batch_size][selected_features]
            batch_df = batch_df.fillna(0)

            # Convert to DMatrix with proper feature names
            feature_names = list(batch_df.columns)  # Convert Index to list of strings
            dmatrix = xgb.DMatrix(batch_df.values, feature_names=feature_names)

            probabilities = model.predict(dmatrix)
            predictions = ['Pathogenic' if p >= 0.5 else 'Benign' for p in probabilities]

            all_predictions.extend(predictions)
            all_probabilities.extend(probabilities)

            progress_bar.progress(min((i + batch_size) / len(input_df), 1.0))

        progress_bar.empty()

        if errors:
            with open('prediction_errors.txt', 'w') as f:
                f.write("\n".join(errors))
            st.markdown("### Download Error Log")
            st.download_button(
                label="📋 Download Error Log",
                data=open('prediction_errors.txt', 'rb'),
                file_name="prediction_errors.txt",
                mime="text/plain",
                use_container_width=True
            )

        return all_predictions, all_probabilities

    except Exception as e:
        errors.append(str(e))
        with open('prediction_errors.txt', 'w') as f:
            f.write("\n".join(errors))
        st.error(f"Error in batch predictions: {str(e)}")
        st.error("Check your CSV for valid data (correct columns, data types).")
        st.download_button(
            label="📋 Download Error Log",
            data=open('prediction_errors.txt', 'rb'),
            file_name="prediction_errors.txt",
            mime="text/plain"
        )
        return [], []

def display_prediction_result(prediction, probability):
    """Display prediction result with enhanced styling"""
    if prediction is None:
        st.error("Prediction failed. Please check your input data.")
        return

    if prediction == "Pathogenic":
        st.markdown(f'''
        <div class="prediction-result pathogenic">
            <strong>⚠️ PATHOGENIC</strong><br>
            XGBoost Confidence: {probability:.2%}
        </div>
        ''', unsafe_allow_html=True)

        st.warning("""
        **Clinical Interpretation:**
        ⚠️ This mutation is predicted to be **PATHOGENIC** with high confidence.
        🔍 Consider further clinical evaluation and functional studies.
        📌 May have significant impact on protein function.
        """)

    elif prediction == "Benign":
        st.markdown(f'''
        <div class="prediction-result benign">
            <strong>✔️ BENIGN</strong><br>
            XGBoost Confidence: {probability:.2%}
        </div>
        ''', unsafe_allow_html=True)

        st.success("""
        **Clinical Interpretation:**
        ✔️ This mutation is predicted to be **BENIGN** with high confidence.
        🧬 Likely represents a neutral polymorphism.
        📌 Low likelihood of clinical significance.
        """)

# Main application
def main():
    st.markdown('<h1 class="main-header">🧬 XGBoost Gene Mutation Pathogenicity Predictor</h1>', unsafe_allow_html=True)

    # Load model artifacts
    model, selected_features, label_encoders, feature_modes = load_model_artifacts()

    if model is None:
        st.stop()

    st.markdown("### 🎯 Predict whether a gene mutation is **Pathogenic** or **Benign** using XGBoost AI")

    # Model info
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown('<div class="metric-card"><h4>🤖 Model</h4><p>XGBoost Classifier</p></div>', unsafe_allow_html=True)
    with col2:
        st.markdown(f'<div class="metric-card"><h4>📊 Features</h4><p>{len(selected_features)} Variables</p></div>', unsafe_allow_html=True)
    with col3:
        st.markdown('<div class="metric-card"><h4>🎯 Accuracy</h4><p>99.99%</p></div>', unsafe_allow_html=True)

    # Add prominent CSV upload section at the top
    st.markdown("""
    <div class="upload-section">
        <h3>📁 Quick CSV Upload for Batch Predictions</h3>
        <p>Upload your gene mutation CSV file to get instant XGBoost pathogenicity predictions for multiple mutations!</p>
    </div>
    """, unsafe_allow_html=True)

    # CSV Upload section
    with st.expander("🚀 **UPLOAD CSV FILE FOR BATCH PREDICTIONS**", expanded=True):
        csv_upload_section(model, selected_features, label_encoders, feature_modes)

    st.markdown("---")

    # Sidebar for navigation
    st.sidebar.title("🧭 Navigation")
    st.sidebar.markdown("**XGBoost Gene Mutation Predictor**")
    page = st.sidebar.selectbox("Choose a page:", [
        "Single Prediction",
        "Advanced Batch Upload",
        "Feature Information",
        "CSV Format Guide",
        "Model Performance"
    ])

    if page == "Single Prediction":
        single_prediction_page(model, selected_features, label_encoders, feature_modes)
    elif page == "Advanced Batch Upload":
        batch_prediction_page(model, selected_features, label_encoders, feature_modes)
    elif page == "CSV Format Guide":
        csv_format_guide_page(selected_features, label_encoders, feature_modes)
    elif page == "Model Performance":
        model_performance_page()
    else:
        feature_information_page(selected_features)

def batch_prediction_page(model, selected_features, label_encoders, feature_modes):
    st.markdown('<h2 class="sub-header">Advanced Batch Mutation Prediction</h2>', unsafe_allow_html=True)

    st.markdown("Upload a CSV file with mutation data for batch prediction with advanced options.")

    uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
            st.success(f"File uploaded successfully! Shape: {df.shape}")

            st.markdown("**Preview of uploaded data:**")
            st.dataframe(df.head())

            batch_size = st.slider("Batch Size for Processing", min_value=100, max_value=1000, value=500, step=100)

            if st.button("🚀 Run Batch Prediction", type="primary"):
                predictions, probabilities = make_batch_predictions(
                    df, model, selected_features, label_encoders, feature_modes, batch_size
                )

                results_df = df.copy()
                results_df['Predicted_Pathogenicity'] = predictions
                results_df['Pathogenic_Probability'] = [f"{p:.4f}" for p in probabilities]

                st.markdown("**Prediction Results:**")
                st.dataframe(results_df, use_container_width=True)

                csv = results_df.to_csv(index=False)
                st.download_button(
                    label="📥 Download Results as CSV",
                    data=csv,
                    file_name="mutation_predictions.csv",
                    mime="text/csv",
                    use_container_width=True
                )

                st.markdown("**Prediction Summary:**")
                col1, col2, col3 = st.columns(3)
                with col1:
                    st.metric("Total Predictions", len(predictions))
                with col2:
                    st.metric("Pathogenic", sum(1 for p in predictions if p == 'Pathogenic'))
                with col3:
                    st.metric("Benign", sum(1 for p in predictions if p == 'Benign'))

        except Exception as e:
            st.error(f"Error processing file: {e}")
            st.info("Please check that your CSV file contains valid data and correct column names.")

if __name__ == "__main__":
    main()

Writing xgboost_app.py


In [6]:
!wget -q -O - ipv4.icanhazip.com
!streamlit run xgboost_app.py & sleep 5 && npx localtunnel --port 8501

34.27.215.14



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.27.215.14:8501

⠙⠹⠸⠼your url is: https://deep-plants-itch.loca.lt
2025-06-01 15:52:21.888 The `use_column_width` parameter has been deprecated and will be removed in a future release. Please utilize the `use_container_width` parameter instead.
────────────────────────── Traceback (most recent call last) ───────────────────────────
  /usr/local/lib/python3.11/dist-packages/streamlit/elements/lib/image_utils.py:286 in  
  image_to_url                                                                          
────────────────────────────────────────────────────────────────────────────────────────
FileNotFoundError: [Errno 2] No such file or directory: 'feature_importance.png'

During handling of the above exception, another exception occurred:

────────────────────────── Traceback (most recent call last) ────────────────